In [24]:
#Load OpenAI API key from .env file
from dotenv import load_dotenv
import os
from openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
#client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client = OpenAI()
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say this is a test  tell me korean"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

This is a test in Korean is "이것은 테스트입니다." (igeoseun teseutimnida).

Text to Speech

https://platform.openai.com/docs/guides/text-to-speech/overview

In [9]:
speech_file_path = "speech2024.mp3"
with client.audio.speech.with_streaming_response.create(
model="tts-1",
voice="alloy",
input="저는 러브탱입니다."
) as response:
        response.stream_to_file(speech_file_path)

https://platform.openai.com/docs/api-reference/images/create <br>
https://platform.openai.com/docs/guides/images/usage

In [15]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
response = client.images.generate(
  model="dall-e-3",
  prompt="a white siamese cat",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url


In [12]:
from IPython.display import Image, display
# 이미지 출력
display(Image(url=image_url))

Speech To Text <br>
https://platform.openai.com/docs/guides/speech-to-text

In [17]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

audio_file= open("speech2024.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

저는 러브탱입니다.


Embeddings <br>
https://platform.openai.com/docs/guides/embeddings/what-are-embeddings  <br>
https://cookbook.openai.com/examples/get_embeddings_from_dataset

In [20]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.embeddings.create(
    input="현재 프랑스에서 올림픽 진행중!",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[-0.027292873710393906, 0.02351832576096058, -0.018761057406663895, -0.017353978008031845, -0.011569319292902946, 0.020570160821080208, 0.005343551281839609, 0.009146016091108322, -0.015276861377060413, -0.03939822316169739, -0.012864725664258003, -0.03676274046301842, 0.0019235668005421758, 0.014104295521974564, 0.006488199345767498, 0.021418875083327293, -0.007409501355141401, 0.05083353444933891, 0.009079012088477612, 0.026823846623301506, 0.034439943730831146, -0.021229030564427376, -0.03611503541469574, 0.0009101346950046718, 0.021095022559165955, -0.0404256135225296, 0.002075721276924014, 0.04243572801351547, 0.007113568019121885, 0.0269355196505785, 0.011010954156517982, -0.027136530727148056, -0.023093968629837036, -0.015310363844037056, 0.012741885147988796, -0.0466792993247509, -0.030397381633520126, 0.021363038569688797, -0.02628781646490097, 0.02724820375442505, 0.012864725664258003, -0.04935945197939873, -0.04393214359879494, 0.0019863827619701624, -0.0026298982556909323, 

In [22]:
import os
from openai import OpenAI
import pandas as pd

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 입력 문장
input_sentence = "현재 프랑스에서 올림픽 진행중!"

# 문장을 개별 단어로 분할
words = input_sentence.split()

# 단어 리스트에 대한 임베딩을 얻는 함수
def get_embeddings(words):
    embeddings = []
    for word in words:
        response = client.embeddings.create(
            input=word,
            model="text-embedding-3-small"
        )
        embeddings.append(response.data[0].embedding)
    return embeddings

# 각 단어에 대한 임베딩을 얻기
word_embeddings = get_embeddings(words)

# 단어와 해당 임베딩을 저장할 DataFrame 생성
df = pd.DataFrame({'Word': words, 'Embedding': word_embeddings})

# DataFrame을 CSV 파일로 저장
df.to_csv('word_embeddings.csv', index=False)

print("임베딩 결과가 word_embeddings.csv 파일에 저장되었습니다.")


임베딩 결과가 word_embeddings.csv 파일에 저장되었습니다.


GPT4 Vision
<br>
https://platform.openai.com/docs/guides/vision

In [23]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image? tell me korean"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이 이미지는 넓은 초원과 나무들, 그리고 그 사이로 이어지는 나무 데크가 있는 평화로운 풍경을 보여줍니다. 푸른 하늘과 흰 구름이 함께 어우러져 자연의 아름다움을 잘 나타내고 있습니다.', role='assistant', function_call=None, tool_calls=None))
